# Projekt MUP
---

#### Wpływ falochronów na prędkość i wysokość fali przy brzegu
---

W projekcie przeprowadzimy symulację fali padającej na brzeg i sprawdzimy jej zachowanie w trzech scenariuszach: bez przeszkód, z pojedynczym oraz z podwójnym falochronem. Porównując wysokość i prędkość fali w poszczególnych przypadkach, ocenimy, jak falochrony wpływają na parametry fali.


# Setup

## Imports

In [ ]:
import numpy as np
import pandas as pd
import scienceplots
from cycler import cycler
from matplotlib import pyplot as plt
from open_atmos_jupyter_utils import show_plot
from src.source import Config

plt.style.use(
    [
        "science",
        "grid",
        "notebook",
        "high-vis",
        # "default",
        # "dark_background",
    ]
)
nice_colors = [
    "royalblue",
    "darkorange",
    "mediumseagreen",
    "crimson",
    "mediumpurple",
    "sienna",
    "hotpink",
    "slategray",
    "goldenrod",
    "turquoise",
]
plt.rcParams["text.usetex"] = False
plt.rcParams["axes.prop_cycle"] = cycler(color=nice_colors)

# Classes

## Solver
Do rozwiązywania równania płytkiej wody użyjemy solver'a z laboratoriów

## Klasa do rysowania wykresów

## Przykładowe geometrie dna

## Config

Powyższe klasy znajdują się w pliku [`src/source.py`](https://github.com/MUPAGH2025/2dybich/blob/main/labs/project/src/source.py) dla oszczędności miejsca i limitu wielkości pliku na GitHub'ie (100MB)

# Description
Stosujemy nazewnictwo parametrów z zajęć

In [ ]:
plt.figure(figsize=(14.5, 3))
plt.axis("off")

y = np.linspace(-np.pi, np.pi)
z = np.cos(y + 2) / 2 + 0.25
b = np.sin(y) / 3 - 2

arrow_kwargs = {"length_includes_head": True, "head_width": 0.05}

ix = 5
color = "brown"
label = "$b$"
plt.fill_between(y, b, -2.75, color=color, label=f"$z=-${label}", alpha=0.5)
plt.arrow(y[ix], 0, 0, b[ix], color=color, **arrow_kwargs)
plt.annotate(f" {label}", xy=(y[ix], b[ix] + 1), color=color)

ix = 10
color = "blue"
label = r"$\zeta$"
plt.fill_between(y, z, b, color=color, label=f"$z=${label}", alpha=0.1)
plt.arrow(y[ix], 0, 0, z[ix], color=color, **arrow_kwargs)
plt.annotate(f" {label}", xy=(y[ix], z[ix] - 0.5), color=color)

ix = 15
color = "black"
label = "$h$"
plt.plot(y, np.full_like(y, 0), linestyle="--", color=color, label="$z=0$")
plt.arrow(y[ix], 0, 0, z[ix], color=color, **arrow_kwargs)
plt.arrow(y[ix], 0, 0, b[ix], color=color, **arrow_kwargs)
plt.annotate(f" {label}", xy=(y[ix], -1), color=color)

plt.legend(loc=10)
show_plot("symbols")

# Simulations

## Test Simulation

### Parameters

Na początku symulujemy falę testową, do której będziemy porównywać wyniki z falochronami. Ustawiamy parametry podane poniżej. Zakładamy wartość $\varepsilon = 0.1$ - jest to wartość głębokości poniżej poziomu morza, dla której uważamy, że przeszkoda wystaje nad powierzchnię (całkowite odbicie). Będziemy się starać unikać wartości $b<\varepsilon$, chyba że będzie to celowe. 

Zaczynamy od sprawdzenia parametrów fali dla dna o charakterystyce liniowej:


\begin{align*}
    b(x) &= ax+c \\
    b(x) &= x+0.5 \\
    z(x) &= -x-0.5 \\
\end{align*}


Jako falę początkową przyjmujemy:
\begin{equation}
    \zeta(x,y) = Ae^{\frac{-(x - x_0)^2}{2\sigma}},
\end{equation}
gdzie: $A = 0.025$, $x_0 = \max(x)$, $\sigma = 0.05$.

Jako że padająca fala nie zależy od $y$, zbadamy zależności tylko w osi $x$, po środku siatki.

Tworzymy DataFrame do przechowywania danych z symulacji:
- `zeta_max` - maksymalna wysokość fali,
- `mag_max` - maksymalna prędkość fali jako długość wektora prędkości,
- `ux_min` - minimalna wartość $x$-owej prędkości fali (minimalna, ponieważ fala porusza się w stronę brzegu - wartości $x$ zmniejszają się - prędkość jest ujemna)

In [ ]:
data = pd.DataFrame(columns=["zeta_max", "mag_max", "ux_min"])

In [ ]:
CONSTANTS = {
    "grid": (500, 100),
    "depth": 0.5,
    "dt_over_dxy": (0.2, 0.05),
    "nt": 1200,
    "eps": 0.1,
    "g": 10,
    "outfreq": 3,
    "n_frames": 100,
}
cfg_test = Config(**CONSTANTS)
data_label = "test"

cfg_test.set_bath("linear")

### Initial

Poniżej można zobaczyć kształt dna oraz falę początkową

In [ ]:
fig = plt.figure()
cfg_test.plots.plot_initial_3d(fig)
fig.tight_layout()
show_plot(f"{data_label}_3D")

### Simulation

In [ ]:
cfg_test.run_sim()

### Results

Na rysunku (a), (c) i (b) poniżej można zobaczyć odpowiednio rozkład prędkości $\upsilon_x$, $|\upsilon|$ oraz $\zeta_x$, a na rysunku (d) rozkład dna.

Widzimy, że wysokość fali $\zeta_x$ jest najwyższa na początku oraz podczas odbicia od brzegu, a pomiędzy tymi punktami utrzymuje się na podobnym poziomie.

Rysunek (a) pokazuje kierunek rozchodzenia się fali: na początku $\upsilon_x$ jest ujemne - ruch w stronę niższych wartości $x$ (w stronę brzegu), a po odbiciu składowa $x$-owa prędkości fali jest dodatnia - fala oddala się od brzegu.

Analizując rysunek (c) można zauważyć, że na początku ruchu, $|\upsilon|$ jest największy - jest to region najbardziej oddalony od brzegu, a co za tym idzie - największą głębokością (rysunek (d)), co skutkuje największą wartością prędkości. Podczas zbliżania się do brzegu, fala zwalnia, ponieważ głębokość maleje, a po odbiciu znowu rośnie.

Próba kontrolna potwierdziła oczekiwania teoretyczne, jak również zależność odwrotności głębokości i prędkości fali.

Dodatkowo zapisujemy maksymalną wysokość oraz prędkość fali w przedziale czasowym $\langle t_0,\, t_1 \rangle$ - obszar 1 (wybranym arbitralnie).

Przyjmujemy nazewnictwo obszarów czasowych:
$
\begin{equation}
    \mathrm{obszar}_n = \langle t_{n-1},\, t_{n} \rangle
\end{equation}
$

In [ ]:
kwargs = {
    "t": {"c": "crimson", "ls": "--", "alpha": 0.6},
    "x": {"c": "pink", "ls": "--", "alpha": 0.6},
}


fig, ax = cfg_test.plots.plot_bath_times()
ts = [50, 220]
for name, ax in ax.items():
    if name == "bathymetry":
        continue
    cfg_test.plots.axvline_t(ax, ts, **kwargs["t"])

cfg_test.get_data_params(ts, data, data_label)
selected = cfg_test.get_selected_data(data, data_label)
print(selected)
show_plot(f"{data_label}_result")

## Pojedynczy próg

### Parameters

Wszystkie parametry pozostawiamy bez zmian, zmieniamy dno - pojedynczy próg.

In [ ]:
cfg_single_step = Config(**CONSTANTS)
data_label = "one_step"
x0 = 0.4
x1 = 0.6
x_mid = 0.5 * (x0 + x1)
A = 0.4
cfg_single_step.set_bath("linear_step", x0=x0, x1=x1, A=A)

### Initial

In [ ]:
fig = plt.figure()
cfg_single_step.plots.plot_initial_3d(fig)
fig.tight_layout()
show_plot(f"{data_label}_3D")

### Simulation

In [ ]:
cfg_single_step.run_sim()

### Results

Dzielimy oś czasu na 3 obszary:
 - obszar 1: przed barierą,
 - obszar 2: tuż przed i tuż po barierze,
 - obszar 3: za barierą.

W tym wypadku nie obserwujemy linii prostych jak poprzednio. Wysokość maksymalna w obszarach 1 i 3 jest mniejsza niż w symulacji testowej o około 7%, natomiast w obszarze 2 jest prawie 22% większa - jest to spowodowane "wybrzuszeniem" fali na barierze, co bardzo dobrze widać na wykresie (b) na wysokości różowej linii przerywanej. Fala "czując" pod sobą przeszkodę, zwiększa swoją wysokość.

Widzimy również, że przed barierą maksymalna prędkość fali nie różni się znacząco od testowej, podczas interakcji z przeszkodą jest ok. 8% mniejsza, a po jej pokonaniu fala spowalnia o aż 14%.

In [ ]:
fig, axes = cfg_single_step.plots.plot_bath_times()
ts = [50, 90, 140, 220]
for name, ax in axes.items():
    if name != "bathymetry":
        cfg_single_step.plots.axvline_t(ax, ts, **kwargs["t"])
        ax.axhline(x_mid, label=r"$\mathrm{barrier}$", **kwargs["x"])
    else:
        ax.axvline(x_mid, label=r"$\mathrm{barrier}$", **kwargs["x"])
    ax.legend(fontsize=12)
cfg_single_step.get_data_params(ts, data, data_label)
selected = cfg_single_step.get_selected_data(data, data_label)
print(selected)
print(f"\nComparison ({data_label}/test):")
print(cfg_single_step.get_compared_data(data, data_label).round(5))
show_plot(f"{data_label}_result")

## Podwójny próg

### Parameters

Parametry takie same, ale dodajemy drugą zaporę.

In [ ]:
cfg_double_step = Config(**CONSTANTS)
data_label = "two_step"
x0 = 0.4
x1 = 0.6
periods = 2
A = 0.4
cfg_double_step.set_bath("linear_step", x0=x0, x1=x1, A=A, periods=periods)

### Initial

In [ ]:
fig = plt.figure()
cfg_double_step.plots.plot_initial_3d(fig)
fig.tight_layout()
show_plot(f"{data_label}_3D")

### Simulation

In [ ]:
cfg_double_step.run_sim()

### Results

Również i w tym przypadku - największa wysokość fali jest podczas spotkania z zaporą, w szczególności jest to zauważalne na rysunku (b), na którym widać dwa małe wybrzuszenia tuż przez wejściem fali na przeszkodę. Maksymalna wysokość fali przed falochronem jest ok. 12% mniejsza niż testowa, natomiast po przejściu przez niego, spada do 81% wartości początkowej.

Rozkład prędkości również wygląda podobnie, jak poprzednio, tylko że widoczny jest większy efekt. Fala, po przejściu przez falochron traci ok. 25% swojej prędkości.

In [ ]:
fig, axes = cfg_double_step.plots.plot_bath_times()
ts = [50, 80, 160, 220]
xs = cfg_double_step.bath_x_peaks
for name, ax in axes.items():
    if name != "bathymetry":
        cfg_double_step.plots.axvline_t(ax, ts, **kwargs["t"])
        for i, x_mid in enumerate(xs):
            ax.axhline(
                x_mid, label=r"$\mathrm{barrier}$" if i == 0 else "", **kwargs["x"]
            )
    else:
        for i, x_mid in enumerate(xs):
            ax.axvline(
                x_mid, label=r"$\mathrm{barrier}$" if i == 0 else "", **kwargs["x"]
            )
    ax.legend(fontsize=12)
cfg_double_step.get_data_params(ts, data, data_label)
selected = cfg_double_step.get_selected_data(data, data_label)
print(selected)
print(f"\nComparison ({data_label}/test):")
print(cfg_double_step.get_compared_data(data,data_label).round(5))
show_plot(f"{data_label}_result")


## Inna dokładność czasu

Mając już wyniki dla stałej siatki i kroku czasowego, porównamy zależność parametrów fali padającej na podwójny falochron z innymi wartościami $\Delta x$ i $\Delta t$. Na początku zmniejszymy dokładność czasu o połowę - oznacza to zwiększenie parametru `dt_over_dxy` dwukrotnie (zmieniamy krok w osi $x$, ponieważ nie zajmujemy się osią $y$).

### Parameters

In [ ]:
cfg_double_step_t = Config(**CONSTANTS)
mul = 0.5
cfg_double_step_t.dt_over_dxy = (
    cfg_double_step_t.dt_over_dxy[0] / mul,
    cfg_double_step_t.dt_over_dxy[1],
)
cfg_double_step_t.nt = int(cfg_double_step_t.nt * mul)
data_label = "two_step_t"
x0 = 0.4
x1 = 0.6
periods = 2
A = 0.4
cfg_double_step_t.set_bath("linear_step", x0=x0, x1=x1, A=A, periods=periods)

### Simulation

In [ ]:
cfg_double_step_t.run_sim()

### Results

Początkowa dokładność czasowa wynosiła `(0.2, 0.05)` i zmieniamy ją na `(0.4, 0.05)`. W związku z tym, należy również dwukrotnie zmniejszyć liczbę kroków czasowych, aby zachować taki sam kształt przebiegu.

Parametry fali zmieniły się nieznacznie. Procentowe wartości względem fali testowej różnią się od poprzedniej dokładności czasowej o ok. 1%, jednakże porównując dane z falą z podwójną przeszkodą wartości zmieniły się o ułamki procenta.

In [ ]:
fig, axes = cfg_double_step_t.plots.plot_bath_times()
ts = [int(mul * n) for n in [50, 80, 160, 220]]
xs = cfg_double_step_t.bath_x_peaks
for name, ax in axes.items():
    if name != "bathymetry":
        cfg_double_step_t.plots.axvline_t(ax, ts, **kwargs["t"])
        for i, x_mid in enumerate(xs):
            ax.axhline(
                x_mid, label=r"$\mathrm{barrier}$" if i == 0 else "", **kwargs["x"]
            )
    else:
        for i, x_mid in enumerate(xs):
            ax.axvline(
                x_mid, label=r"$\mathrm{barrier}$" if i == 0 else "", **kwargs["x"]
            )
    ax.legend(fontsize=12)
cfg_double_step_t.get_data_params(ts, data, data_label)
selected = cfg_double_step_t.get_selected_data(data, data_label)
print(f"Grid: {cfg_double_step.grid} -> {cfg_double_step_t.grid}")
print(f"Time: {cfg_double_step.dt_over_dxy} -> {cfg_double_step_t.dt_over_dxy}")
print(f"Time steps: {cfg_double_step.nt} -> {cfg_double_step_t.nt}")
print()
print(selected)
print(f"\nComparison ({data_label}/test):")
print(cfg_double_step_t.get_compared_data(data, data_label).round(5))
print(f"\nComparison ({data_label}/two_step):")
print(cfg_double_step_t.get_compared_data(data, data_label, "two_step").round(5))
show_plot(f"{data_label}_result")

## Inna dokładność siatki

Wracamy do poprzednich wartości parametrów, ale tym razem zmienimy rozdzielczość siatki - zmniejszymy liczbę punktów siatki o połowę.

### Parameters

In [ ]:
cfg_double_step_x = Config(**CONSTANTS)
mul = 1 / 2
cfg_double_step_x.grid = (
    int(cfg_double_step_x.grid[0] * mul),
    int(cfg_double_step_x.grid[1] * mul),
)
cfg_double_step_x.nt = int(cfg_double_step_x.nt * mul)
data_label = "two_step_x"
x0 = 0.4
x1 = 0.6
periods = 2
A = 0.4
cfg_double_step_x.set_bath("linear_step", x0=x0, x1=x1, A=A, periods=periods)

### Simulation

In [ ]:
cfg_double_step_x.run_sim()

### Results

W tym przypadku porównanie wartości symulacji z wartościami z oryginalną siatką pokazuje większe różnice niż przy zmianie dokładności czasu. Teraz, zamiast zmian o ułamki procenta, dostajemy kilkuprocentowe różnice parametrów fali, ale tylko po pokonaniu dwóch barier. W obszarze 1 i 2 wahania wynoszą ok 0.2%, natomiast w obszarze trzecim - 7.5%.

Pomimo gorszej jakości siatki i wyników, fala do brzegu przypływa o około 17% niższa i 18% wolniejsza.

In [ ]:
fig, axes = cfg_double_step_x.plots.plot_bath_times()
ts = [int(n * mul) for n in [50, 80, 160, 220]]
xs = cfg_double_step_x.bath_x_peaks
for name, ax in axes.items():
    if name != "bathymetry":
        cfg_double_step_x.plots.axvline_t(ax, ts, **kwargs["t"])
        for i, x_mid in enumerate(xs):
            ax.axhline(
                x_mid, label=r"$\mathrm{barrier}$" if i == 0 else "", **kwargs["x"]
            )
    else:
        for i, x_mid in enumerate(xs):
            ax.axvline(
                x_mid, label=r"$\mathrm{barrier}$" if i == 0 else "", **kwargs["x"]
            )
    ax.legend(fontsize=12)
cfg_double_step_x.get_data_params(ts, data, data_label)
selected = cfg_double_step_x.get_selected_data(data, data_label)
print(f"Grid: {cfg_double_step.grid} -> {cfg_double_step_x.grid}")
print(f"Time: {cfg_double_step.dt_over_dxy} -> {cfg_double_step_x.dt_over_dxy}")
print(f"Time steps: {cfg_double_step.nt} -> {cfg_double_step_x.nt}")
print()
print(selected)
print(f"\nComparison ({data_label}/test):")
print(cfg_double_step_x.get_compared_data(data, data_label).round(5))
print(f"\nComparison ({data_label}/two_step):")
print(cfg_double_step_x.get_compared_data(data, data_label, "two_step").round(5))
show_plot(f"{data_label}_result")

# Podsumowanie

Tabela poniżej przedstawia porównanie różnicy procentowej $\frac{\mathrm{sim}}{\mathrm{test}}-1$ wyrażonej w procentach.

Widzimy, że w obszarze pierwszym maksymalna wysokość fali jest zwykle około 10% mniejsza niż fala testowa, a prędkości fali są porównywalne.

W obszarze, w którym fala spotyka przeszkodę dostajemy ok. 20% wyższą falę, która "czuje" działanie podwyższonego terenu, jednocześnie zwalniając o około 5%.

W ostatnim obszarze, fala, docierając do brzegu, w każdym badanym przypadku jest mniejsza oraz wolniejsza. Pojedyncza bariera zmniejszyła falę o 5% oraz spowolniła ją o 14%. Dodanie kolejnego wybrzuszenia spowodowało znaczne wzmocnienie tych efektów, skutkując zmniejszeniem i spowolnieniem fali o około 20%.

Zmiana kroku czasowego nie wpłynęła zbytnio na różnice w wysokości czy prędkości fali (w porównaniu do takich samych warunków z niezmienionym krokiem czasowym) - zmiany pozostawały na granicach ułamków procentów. Z drugiej strony, zmiany dokładności siatki zaniżyły efekty falochronów, pogarszając ich efektywność - zamiast obniżenia wysokości fali o 20%, mamy 17% oraz redukcja prędkości zamiast 24% wynosiła tylko 18%.

Przedstawione wyniki oraz symulację potwierdzają słuszność obecności falochronów. Dzięki ich obecności można redukować efekty burzliwego morza, dbając o bezpieczeństwo ludzi przebywających w jego pobliżu, jedynie modyfikując strukturę dna.

In [ ]:
perc_comp = ((data.iloc[1:].divide(data.iloc[0]) - 1) * 100).round(2)
print("Obszar 1:")
print(perc_comp[perc_comp.index.str.contains("_0")])
print("\nObszar 2:")
print(perc_comp[perc_comp.index.str.contains("_1")])
print("\nObszar 3:")
print(perc_comp[perc_comp.index.str.contains("_2")])